In [1]:
import numpy as np
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import pandas as pd
import nltk

C:\Users\mayan\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
fname = 'wordVec.bin'
word_vectors = KeyedVectors.load(fname)

In [3]:
data = pd.read_json('corpus.json')

In [4]:
data = data['Quote'].unique()

In [5]:
data = pd.DataFrame({'Quote':data})

In [6]:
data.head()

,Quote
0,"Don't cry because it's over, smile because it ..."
1,"I'm selfish, impatient and a little insecure. ..."
2,Be yourself; everyone else is already taken.
3,Two things are infinite: the universe and huma...
4,"Be who you are and say what you feel, because ..."


In [7]:
X = []
Y = []
for index,row in data.iterrows():
    
    cur_row = nltk.word_tokenize(row['Quote'])
    
    if len(cur_row) > 8:
        x = cur_row[:-1]
        x.insert(0,'STARTPAD')
        X.append(x)
        Y.append(cur_row)
    
        if index%5000 == 0 :
            print("--Processed : " + str(index) + " sentences--")

--Processed : 0 sentences--
--Processed : 5000 sentences--
--Processed : 10000 sentences--
--Processed : 15000 sentences--
--Processed : 20000 sentences--
--Processed : 25000 sentences--
--Processed : 30000 sentences--
--Processed : 35000 sentences--


In [8]:
#Convert X and Y to their word embeddings 

x_data = []
y_data = []

for i in range(len(X)):
    temp_x = []
    temp_y = []
    
    for j in range(len(X[i])):
        
        temp_x.append(word_vectors[X[i][j]])
        temp_y.append(word_vectors[Y[i][j]])
    x_data.append(np.array(temp_x))
    y_data.append(np.array(temp_y))
    
    if i%5000 == 0 :
            print("--Processed : " + str(i) + " sentences--")

--Processed : 0 sentences--
--Processed : 5000 sentences--
--Processed : 10000 sentences--
--Processed : 15000 sentences--
--Processed : 20000 sentences--
--Processed : 25000 sentences--
--Processed : 30000 sentences--
--Processed : 35000 sentences--


In [9]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [10]:
#Padding

from keras.preprocessing.sequence import pad_sequences

max_len = 25

x_data = pad_sequences(maxlen=max_len, sequences=x_data, padding="post", value=word_vectors['ENDPAD'],truncating='post')
y_data = pad_sequences(maxlen=max_len, sequences=y_data, padding="post", value=word_vectors['ENDPAD'],truncating='post')

C:\Users\mayan\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

In [12]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(28237, 25, 100) (28237, 25, 100)
(7060, 25, 100) (7060, 25, 100)


In [47]:
#Building the model

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Flatten,Dense, TimeDistributed, Dropout,Bidirectional
from keras.layers.advanced_activations import LeakyReLU, PReLU

model = Sequential()
model.add(Bidirectional(LSTM(300,return_sequences=True),input_shape=(25,100)))
model.add(TimeDistributed(Dense(200)))
model.add(LeakyReLU(0.35))
model.add(TimeDistributed(Dense(100,activation='relu')))

In [48]:
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])

In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 25, 600)           962400    
_________________________________________________________________
time_distributed_17 (TimeDis (None, 25, 200)           120200    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 25, 200)           0         
_________________________________________________________________
time_distributed_18 (TimeDis (None, 25, 100)           20100     
Total params: 1,102,700
Trainable params: 1,102,700
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.fit(x_train, y_train, batch_size=5, epochs=10, validation_split=0.1, verbose=1)

Train on 25413 samples, validate on 2824 samples
Epoch 1/10
 7740/25413 [========>.....................] - ETA: 8:15 - loss: 0.3458 - acc: 0.4108

KeyboardInterrupt: 